## Предобработка признаков курсов

In [166]:
import pandas as pd
import json
import numpy as np
import re
import gensim
import nltk 
from nltk.tokenize import sent_tokenize,word_tokenize
from nltk.corpus import stopwords
import pymorphy2
from gensim.models import Word2Vec
import spacy
import ru_core_news_md
from sklearn.feature_extraction.text import TfidfVectorizer
import gensim.downloader as download_api

In [103]:
with open('./hse_dpo_portal_data.json') as json_file:
    courses_features = pd.DataFrame(json.load(json_file)['data'])

In [104]:
df = pd.read_csv('./data_t_row_6m.csv')

Посмотрим сколько пропусков в столбцах

In [105]:
courses_features.isna().sum()

id                                 0
url                                0
title                              0
description                        6
duration                           6
type                               0
study_format                       4
org_unit_id                        3
org_unit_title                     3
external_application_form_url    235
application_form_url               0
external_ask_form_url            276
ask_form_url                       0
related_programs                 329
dtype: int64

Посмотрим, есть ли дубликаты

In [106]:
courses_features.drop(['related_programs'],axis = 1).duplicated().sum()

0

Работа со столбцом duration

In [107]:
courses_features.duration.unique()

array(['3 недели', '13 месяцев', '6 месяцев', '26 недель', '4 недели',
       '8 недель', '2 месяца', '10 недель', '12 месяцев', '19 недель',
       '8 месяцев', '7 недель', '4 месяца', '1 неделя', '3 дня', '6 дней',
       '12 недель', '8 дней', '17 дней', '9 дней', '9 недель', '6 недель',
       '9.5 недель', '9 месяцев', '5 недель', '24 месяца', '7 месяцев',
       '3 месяца', '5 дней', '2 недели', '10 дней', '4 дня', '7 дней',
       None, '4.5 месяца', '18 месяцев', '35 недель', '2 дня',
       '13 недель', '12 дней', '26 дней', '1 месяц', '5 месяцев',
       '15 недель', '66 дней', '16 месяцев', '14 месяцев', '11 месяцев',
       '10 месяцев', '13 дней', '15 месяцев', '1.5 месяц', '2.5 недели',
       '19 месяцев', '33 недели', '2.5 месяца', '41 неделя', '17 месяцев',
       '22 недели', '18 недель', '25 недель', '11 дней', '20 дней',
       '17 недель', '40 недель', '14 недель'], dtype=object)

In [108]:
duration_str = ['дн','недел','месяц','год','лет']
duration_convert_coef = [1, 7, 30, 365, 365]

def convert_to_days(duration):
    if duration == None:
        return np.NaN
    duration_value, duration_type = duration.split()
    duration_value = float(duration_value)
    duration_value = sum([(duration_str[i] in duration_type) * duration_value * duration_convert_coef[i] for i in range(len(duration_str))])
    return duration_value

courses_features['duration_days'] = courses_features['duration'].apply(convert_to_days)

In [109]:
courses_features[courses_features['duration'].isna()]

,id,url,title,description,duration,type,study_format,org_unit_id,org_unit_title,external_application_form_url,application_form_url,external_ask_form_url,ask_form_url,related_programs,duration_days
101,486202666,https://www.hse.ru/edu/dpo/486202666,"Режиссура кино, телевидения и цифровых медиапр...",Вы будете учиться у мастера курса – режиссера ...,None,Профессиональная переподготовка,Очный,119384956.0,Факультет креативных индустрий,None,https://www.hse.ru/polls/486299509.html,None,https://www.hse.ru/polls/486299689.html,None,NaN
213,777605358,https://www.hse.ru/edu/dpo/777605358,Практика правоприменения норм Федерального зак...,Курс повышения квалификации выстроен в соответ...,None,Повышение квалификации,Онлайн,135339.0,Региональный центр подготовки кадров для систе...,None,https://www.hse.ru/polls/777605174.html,None,https://www.hse.ru/polls/777605310.html,"[486199682, 486199693, 486200020, 486200022, 7...",NaN
237,480736506,https://www.hse.ru/edu/dpo/480736506,Организация строительного производства. Управл...,Слушатели совершенствуют и получат новые компе...,None,Повышение квалификации,Смешанный,61217348.0,Институт строительства и жилищно-коммунального...,,https://www.hse.ru/polls/480736491.html,,https://www.hse.ru/polls/480736500.html,None,NaN
378,480508962,https://www.hse.ru/edu/dpo/480508962,Организация строительного производства. Управл...,Слушатели совершенствуют и получат новые компе...,None,Повышение квалификации,Дистанционный,61217348.0,Институт строительства и жилищно-коммунального...,,https://www.hse.ru/polls/480508964.html,,https://www.hse.ru/polls/480508973.html,None,NaN
409,472495888,https://www.hse.ru/edu/dpo/472495888,Ценообразование и сметное дело,В процессе обучения слушатели будут ознакомле...,None,Профессиональная переподготовка,Дистанционный,61217348.0,Институт строительства и жилищно-коммунального...,,https://www.hse.ru/polls/472495873.html,,https://www.hse.ru/polls/472495882.html,None,NaN
433,480746222,https://www.hse.ru/edu/dpo/480746222,Эксплуатация зданий и сооружений,Программы разработана с применением актуальных...,None,Профессиональная переподготовка,Смешанный,61217348.0,Институт строительства и жилищно-коммунального...,,https://www.hse.ru/polls/480746224.html,,https://www.hse.ru/polls/480746233.html,None,NaN


Проверим, есть ли несколько строк для одного курса

In [110]:
courses_features['id'].duplicated().sum()

0

Посмотрим на столбец type

In [111]:
courses_features['type'].value_counts()

Повышение квалификации                     455
Профессиональная переподготовка            164
Specialized Master                          10
Дополнительное образование для взрослых      6
MBA                                          4
Name: type, dtype: int64

Разберемся со столбцом org_unit_id

In [112]:
courses_features['org_unit_id'].isna().sum()

3

Проверим, что для нас этот столбец не несет дополнительной информации, так как название организации мы уже знаем

In [113]:
(courses_features.groupby('org_unit_id').agg({'org_unit_title':'nunique'}) != 1).sum()

org_unit_title    0
dtype: int64

Он действительно не несет дополнительную информацию.

In [114]:
courses_features = courses_features.drop(['org_unit_id'],axis = 1)

Разберемся со всеми столбцами ссылок и посмотрим используются ли они при заполнении форм пользователями в нашем датасете.

In [115]:
courses_features.head()

,id,url,title,description,duration,type,study_format,org_unit_title,external_application_form_url,application_form_url,external_ask_form_url,ask_form_url,related_programs,duration_days
0,484817118,https://www.hse.ru/edu/dpo/484817118,Управление государственными и муниципальными з...,Участие в обучении представителей регулятора и...,3 недели,Повышение квалификации,Смешанный,Институт по ресурсному обеспечению управления ...,,https://www.hse.ru/polls/484817103.html,,https://www.hse.ru/polls/484817112.html,None,21.0
1,486198673,https://www.hse.ru/edu/dpo/486198673,Экономика и управление недвижимостью,Программа полностью разработана специалистами ...,13 месяцев,Профессиональная переподготовка,Смешанный,Институт налогового менеджмента и экономики не...,None,https://www.hse.ru/polls/486295892.html,None,https://www.hse.ru/polls/486296072.html,"[474421659, 486198644]",390.0
2,430319050,https://www.hse.ru/edu/dpo/430319050,HR-менеджмент,Онлайн-курс направлен на формирование и развит...,6 месяцев,Профессиональная переподготовка,Онлайн,Бизнес-школа,None,https://www.hse.ru/polls/430319052.html,None,https://www.hse.ru/polls/430319061.html,"[410026421, 424087715, 430128038, 495257336, 5...",180.0
3,463184567,https://www.hse.ru/edu/dpo/463184567,Бухгалтер коммерческой организации,Соответствие программы требованиям профстандар...,26 недель,Профессиональная переподготовка,Дистанционный,Центр развития компетенций в области финансов ...,None,https://www.hse.ru/polls/463184569.html,,https://www.hse.ru/polls/463184578.html,None,182.0
4,420043663,https://www.hse.ru/edu/dpo/420043663,Практический маркетинг,Очный курс по маркетингу поможет освоить базов...,6 месяцев,Профессиональная переподготовка,Очный,Бизнес-школа,None,https://www.hse.ru/polls/420043665.html,None,https://www.hse.ru/polls/420043674.html,"[428429022, 434015991, 522634060, 560610407, 6...",180.0


In [116]:
print("Количество пропусков в столбце external_application_form_url = ",courses_features.external_application_form_url.isna().sum())
print("Количество пропусков в столбце application_form_url = ",courses_features.application_form_url.isna().sum())
print("Количество пропусков в столбце external_ask_form_url = ",courses_features.external_ask_form_url.isna().sum())
print("Количество пропусков в столбце ask_form_url = ",courses_features.ask_form_url.isna().sum())

Количество пропусков в столбце external_application_form_url =  235
Количество пропусков в столбце application_form_url =  0
Количество пропусков в столбце external_ask_form_url =  276
Количество пропусков в столбце ask_form_url =  0


In [117]:
courses_features[~(courses_features.external_application_form_url.isna())][['external_application_form_url','application_form_url']]

,external_application_form_url,application_form_url
0,,https://www.hse.ru/polls/484817103.html
5,,https://www.hse.ru/polls/428429026.html
6,,https://www.hse.ru/polls/484652197.html
7,https://www.hse.ru/polls/663293914.html,https://www.hse.ru/polls/486296961.html
8,,https://www.hse.ru/polls/480913814.html
...,...,...
631,https://www.hse.ru/polls/unit/205756164/476497...,https://www.hse.ru/polls/465142199.html
632,https://www.hse.ru/polls/unit/205756164/476496...,https://www.hse.ru/polls/465141493.html
633,https://www.hse.ru/polls/unit/205756164/476497...,https://www.hse.ru/polls/465142105.html
634,https://www.hse.ru/polls/unit/205756164/476497...,https://www.hse.ru/polls/459526386.html


В столбце external_application_form_url были обнаружены пустые строки, заменим их на None, так как это по сути пропуски

In [118]:
courses_features['external_application_form_url'] = courses_features['external_application_form_url'].apply(lambda x: x if (x != None) and (len(x)>0) else None)

In [119]:
courses_features[~(courses_features.external_application_form_url.isna())][['external_application_form_url','application_form_url']]

,external_application_form_url,application_form_url
7,https://www.hse.ru/polls/663293914.html,https://www.hse.ru/polls/486296961.html
11,https://cs.hse.ru/dpo/polls/407535947.html,https://www.hse.ru/polls/557703978.html
12,https://cs.hse.ru/dpo/polls/409921890.html,https://www.hse.ru/polls/555524329.html
13,https://cs.hse.ru/dpo/polls/409921321.html,https://www.hse.ru/polls/470867464.html
19,https://finance.hse.ru/analytical-reports/poll...,https://www.hse.ru/polls/486303025.html
...,...,...
631,https://www.hse.ru/polls/unit/205756164/476497...,https://www.hse.ru/polls/465142199.html
632,https://www.hse.ru/polls/unit/205756164/476496...,https://www.hse.ru/polls/465141493.html
633,https://www.hse.ru/polls/unit/205756164/476497...,https://www.hse.ru/polls/465142105.html
634,https://www.hse.ru/polls/unit/205756164/476497...,https://www.hse.ru/polls/459526386.html


Как мы видим, ссылки разные, но во всех этих ссылках есть слово polls, посмотрим во всех ли ссылках на форму есть это слово.

In [120]:
print("Количество ссылок external_application_form_url без polls = ",len(courses_features[courses_features['external_application_form_url'].apply(lambda x: (x != None) and ('poll' not in x))]))
print("Количество ссылок application_form_url без polls = ", len(courses_features[courses_features['application_form_url'].apply(lambda x: (x != None) and ('poll' not in x))]))

Количество ссылок external_application_form_url без polls =  20
Количество ссылок application_form_url без polls =  0


In [121]:
print("Количество полных совпадений ссылок application_form_url" ,courses_features['application_form_url'].apply(lambda x: x in df.current_location).sum())
print("Количество полных совпадений ссылок external_application_form_url" ,courses_features['external_application_form_url'].apply(lambda x: x in df.current_location).sum())

Количество полных совпадений ссылок application_form_url 0
Количество полных совпадений ссылок external_application_form_url 0


Все-таки в подавляющем большинстве ссылок есть слово polls, проверим есть ли такие ссылки в датасете с взаимодействиями

In [122]:
print(df.current_location.dropna().apply(lambda x: 'polls' in x).sum())

6


Таким образом, так как у нас практически нет таких ссылок во взаимодействиях, то мы все так же не можем отличать тип формы, которую заполнил пользователь, удалим эти данные из признаков

In [123]:
courses_features = courses_features.drop(['application_form_url','external_application_form_url'],axis = 1)

Проверим нужны ли нам признаки external_ask_form_url и ask_form_url

In [124]:
print("Количество ссылок ask_form_url без слова poll = ",len(courses_features[courses_features['ask_form_url'].apply(lambda x: (x != None) and (x!='') and ('poll' not in x))]))
print("Количество ссылок external_ask_form_url без слова poll = ",len(courses_features[courses_features['external_ask_form_url'].apply(lambda x: (x != None) and (x!='') and ('poll' not in x))]))

Количество ссылок ask_form_url без слова poll =  0
Количество ссылок external_ask_form_url без слова poll =  0


Как мы видим, эти признаки нам не нужны, удалим их

In [125]:
courses_features = courses_features.drop(['ask_form_url'],axis = 1)

In [126]:
courses_features = courses_features.drop(['external_ask_form_url'],axis = 1)

Рассмотрим столбец related_programs

In [127]:
print('Количество пропусков в столбце related_programs = ', courses_features['related_programs'].isna().sum())

Количество пропусков в столбце related_programs =  329


Половина курсов не имеют связанных курсов

In [128]:
courses_features['related_programs'].dropna().apply(lambda x: len(x)).value_counts()

3     194
2      36
1      21
4      20
8      16
5       9
7       6
6       6
9       1
11      1
Name: related_programs, dtype: int64

In [129]:
courses_features[~courses_features['related_programs'].isna()]

,id,url,title,description,duration,type,study_format,org_unit_title,related_programs,duration_days
1,486198673,https://www.hse.ru/edu/dpo/486198673,Экономика и управление недвижимостью,Программа полностью разработана специалистами ...,13 месяцев,Профессиональная переподготовка,Смешанный,Институт налогового менеджмента и экономики не...,"[474421659, 486198644]",390.0
2,430319050,https://www.hse.ru/edu/dpo/430319050,HR-менеджмент,Онлайн-курс направлен на формирование и развит...,6 месяцев,Профессиональная переподготовка,Онлайн,Бизнес-школа,"[410026421, 424087715, 430128038, 495257336, 5...",180.0
4,420043663,https://www.hse.ru/edu/dpo/420043663,Практический маркетинг,Очный курс по маркетингу поможет освоить базов...,6 месяцев,Профессиональная переподготовка,Очный,Бизнес-школа,"[428429022, 434015991, 522634060, 560610407, 6...",180.0
5,428429022,https://www.hse.ru/edu/dpo/428429022,Управление проектами,Краткосрочная программа ДПО охватывает практик...,4 недели,Повышение квалификации,Онлайн,Бизнес-школа,"[408271699, 410026421, 434014480, 522634060, 5...",28.0
7,486199682,https://www.hse.ru/edu/dpo/486199682,"Контрактная система в сфере закупок товаров, р...",Партнеры курса ведут занятия и делают запрос н...,3 недели,Повышение квалификации,Дистанционный,Региональный центр подготовки кадров для систе...,"[470690129, 486199693, 486200020, 486200022, 6...",21.0
...,...,...,...,...,...,...,...,...,...,...
633,465142120,https://www.hse.ru/edu/dpo/465142120,Подтверждение соответствия пищевой продукции ж...,"направлена на формирование новых (углубленных,...",13 недель,Профессиональная переподготовка,Дистанционный,Международный научно-образовательный центр тех...,"[459392511, 465141918, 465142214]",91.0
634,459526384,https://www.hse.ru/edu/dpo/459526384,Подтверждение соответствия продукции легкой пр...,"направлена на формирование новых (углубленных,...",12 недель,Профессиональная переподготовка,Дистанционный,Международный научно-образовательный центр тех...,"[465321492, 465322050, 465323562]",84.0
635,463382443,https://www.hse.ru/edu/dpo/463382443,Главный бухгалтер коммерческой организации,Соответствие программы требованиям профстандар...,40 недель,Профессиональная переподготовка,Дистанционный,Центр развития компетенций в области финансов ...,[463184567],280.0
636,471346702,https://www.hse.ru/edu/dpo/471346702,Машинное обучение,None,14 недель,Повышение квалификации,Очный,Факультет компьютерных наук,"[470867462, 471339557, 471346702, 472490501, 4...",98.0


Так как наша модель будет использовать признаки курсов, в том числе для того чтобы опираться на метод близости курсов, следовательно дополнительные знания сверху о схожести каких-то курсов излишни, особенно учитывая что они имеют много пропусков.

Избавимся от столбцов url, duration

In [130]:
courses_features = courses_features.drop(['url','duration','related_programs'],axis = 1)

Применим для категориальных признаков - type, study_format one-hot-encoding, org_unit_title

In [131]:
courses_features.isna().sum()

id                0
title             0
description       6
type              0
study_format      4
org_unit_title    3
duration_days     6
dtype: int64

In [132]:
from sklearn.preprocessing import OneHotEncoder
cat_columns = ['type','study_format','org_unit_title']
encoder = OneHotEncoder(handle_unknown='ignore',sparse = False)
categorical_features = pd.DataFrame(encoder.fit_transform(courses_features[cat_columns]))
categories = encoder.categories_
for i in range(len(categories)):
    categories[i][categories[i] == None] = 'Unknown_'+cat_columns[i]
categorical_features.columns = np.concatenate(categories)

In [133]:
courses_features = courses_features.drop(['type','study_format'],axis = 1)

In [134]:
courses_features = courses_features.drop(['org_unit_title'],axis = 1)

In [135]:
courses_features = courses_features.join(categorical_features)

Заполним пропуски в колонке с длительностью 0

In [136]:
courses_features['duration_days'] = courses_features['duration_days'].fillna(0)

In [137]:
# from transformers import AutoModel, AutoTokenizer 

# model_name = "sberbank-ai/sbert_large_nlu_ru" 

# model = AutoModel.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# inputs = tokenizer("Hello world!", return_tensors="pt")

# outputs = model(**inputs)

In [138]:
# len(outputs.pooler_output.tolist()[0])

In [139]:
# courses_features.to_csv('./courses_features_1.csv',sep='^')

Сделаем препроцессинг наших текстовых данных для построения векторов для текстовых полей

Для начала приведем все к нижнему регистру

In [140]:
courses_features[['description','title']] = courses_features[['description','title']].apply(lambda x: x.str.lower())

Далее удалим все не буквенно-цифровые символы, а также все знаки препинания и цифры, заменим все пробельные символы пробелами

In [141]:
def clear_text(x):
    if x == None:
        return None
    return re.sub(r'[\s]+',' ',re.sub(r'[_\d]',' ',re.sub(r'[^\w\s.]',' ',x))).strip()
courses_features['description'] = courses_features['description'].apply(clear_text)
courses_features['title'] = courses_features['title'].apply(clear_text)

Далее разобьем текст на токены и удалим стоп-слова

In [142]:
nltk.download('punkt')
nltk.download('stopwords')


def tokenize(x):
    if x == None:
        return []
    tokens = []
    for sentence in sent_tokenize(x, language="russian"):
        sentence = re.sub(r'\.?\.?\.?$','',sentence)
        temp = []
        for token in word_tokenize(sentence, language="russian"):
            if token not in stopwords.words("russian"):
                temp.append(token)
        if len(temp) > 0:
            tokens.append(temp)
    return tokens

courses_features['description'] = courses_features['description'].apply(tokenize)
courses_features['title'] = courses_features['title'].apply(tokenize)

[nltk_data] Downloading package punkt to /home/ujcgjlm/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ujcgjlm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Приведем слова к начальной форме 

In [144]:
morph = pymorphy2.MorphAnalyzer()

def lemmatization(x):
    if x == None:
        return None
    for j in range(len(x)):
        for i in range(len(x[j])):
            x[j][i] = morph.parse(x[j][i])[0].normal_form
    return x

courses_features['description'] = courses_features['description'].apply(lemmatization)
courses_features['title'] = courses_features['title'].apply(lemmatization)

In [ ]:
#Обрежем слова до их основы
# from nltk.stem import SnowballStemmer

# stemmer = SnowballStemmer(language="russian")
# def stemming(x):
#     if x == None:
#         return None
#     for j in range(len(x)):
#         for i in range(len(x[j])):
#             x[j][i] = stemmer.stem(x[j][i])
#     return x 

# courses_features['description'] = courses_features['description'].apply(stemming)
# courses_features['title'] = courses_features['title'].apply(stemming)

Сольем все списки в список для обучения, удалим одинаковые предложения

In [145]:
temp_description = courses_features['description'].sum()
temp_title = courses_features['title'].sum()

In [146]:
def get_unique(temp):
    train = []
    strings = []
    for i in range(len(temp)):
        if len(temp[i]) == 0:
            continue
        s = ""
        if s.join(temp[i]) not in strings:
            strings.append(s.join(temp[i]))
            train.append(temp[i])
    return train

train_description = get_unique(temp_description)
train_title = get_unique(temp_title)

In [150]:
#Обучим модели для каждого признака отдельно
#model_description = Word2Vec(train_description, vector_size=12, min_count=1, workers = 1)
#model_title = Word2Vec(train_title, vector_size=10, min_count=1,workers = 1)

In [159]:
#model_description.wv.most_similar(['медицинский'],topn=4)

[('проблемный', 0.8628553152084351),
 ('полный', 0.8559524416923523),
 ('история', 0.7999520301818848),
 ('база', 0.7975132465362549)]

Данных для выявления четких зависимостей мало, поэтому попробуем использовать уже обученную модель в таком случае использовать стемминг не нужно, и следует добавить часть речи

In [161]:
nlp = ru_core_news_md.load()
def add_tags_1(x):
    for i in range(len(x)):
        for j in range(len(x[i])):
            x[i][j] = x[i][j] + '_' + nlp(x[i][j])[0].pos_
    return x

In [162]:
courses_features['description'] = courses_features['description'].apply(add_tags_1)
courses_features['title'] = courses_features['title'].apply(add_tags_1)

In [163]:
list_sentences_description = list(map(lambda x: ' '.join(x), train_description))
list_sentences_title = list(map(lambda x: ' '.join(x), train_title))

In [165]:
tfidf_description = TfidfVectorizer()
tfidf_title = TfidfVectorizer()
tfidf_unit = TfidfVectorizer()
tfidf_description.fit(list_sentences_description)
tfidf_title.fit(list_sentences_title)
tfidf_dict = {'description':tfidf_description,'title':tfidf_title}

In [167]:
russian_model = download_api.load('word2vec-ruscorpora-300')

[======--------------------------------------------] 13.8% 27.5/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[================----------------------------------] 32.6% 64.8/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[=========================-------------------------] 50.2% 99.8/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[===================================---------------] 70.3% 139.7/198.8MB downloaded

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



[==================================================] 100.0% 198.8/198.8MB downloaded


In [174]:
def get_vector(x,column):
    temp = []
    tfidf = tfidf_dict[column]
    for i in range(len(x)):
        for j in range(len(x[i])):
            if (x[i][j] in russian_model) and (str.lower(x[i][j]) in tfidf.vocabulary_.keys()):
                idf = tfidf.idf_[tfidf.vocabulary_[str.lower(x[i][j])]]
                temp.append(russian_model[x[i][j]] * idf)
    if len(temp) == 0:
        return np.zeros(300)
    return np.mean(temp,axis = 0)

In [175]:
courses_features['title'] = courses_features['title'].apply(get_vector,args=('title',))
courses_features['description'] = courses_features['description'].apply(get_vector,args=('description',))

In [ ]:
for column in ['title','description']:
    for i in range(300):
        courses_features[f'{column}_{i}'] = courses_features[column].apply(lambda x: x[i])

In [177]:
courses_features.head(5)

,id,title,description,duration_days,MBA,Specialized Master,Дополнительное образование для взрослых,Повышение квалификации,Профессиональная переподготовка,Дистанционный,...,description_290,description_291,description_292,description_293,description_294,description_295,description_296,description_297,description_298,description_299
0,484817118,"[0.07838695, -0.17113255, -0.13548853, -0.0729...","[0.084152415, -0.10514774, -0.1096427, 0.04016...",21.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.181073,0.181699,-0.057202,-0.121079,-0.054191,-0.376442,0.214094,-0.054375,-0.155563,-0.158932
1,486198673,"[-0.09847188, -0.13868067, -0.1692385, 0.16344...","[0.02347232, -0.08187603, -0.12719981, 0.16975...",390.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.030400,0.052299,-0.155433,-0.005277,-0.064516,-0.227874,0.201132,-0.052335,-0.246417,-0.090042
2,430319050,"[-0.16362506, -0.18743968, -0.021664428, -0.01...","[0.018166587, -0.08169377, -0.14943294, 0.1871...",180.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.069049,0.067199,-0.060811,-0.057947,-0.055386,-0.167582,0.212634,0.128243,-0.194125,-0.182664
3,463184567,"[0.0019240677, -0.25128558, -0.36541083, 0.153...","[-0.033236083, -0.042782255, -0.1631944, 0.165...",182.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.095487,0.140341,-0.370753,0.049597,-0.212015,-0.250834,0.060476,-0.080062,-0.380524,-0.083958
4,420043663,"[-0.11421541, -0.032873806, 0.09271251, 0.1122...","[-0.052191533, -0.056993086, -0.058620494, 0.0...",180.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.161881,0.017670,-0.034503,-0.115145,-0.055985,-0.169059,0.238166,-0.002234,-0.292364,-0.097465


In [178]:
courses_features = courses_features.drop(['description','title'],axis = 1)

In [ ]:
courses_features.to_csv('./courses_features_2.csv',sep='^')